In [1]:
from dotenv import load_dotenv
import os, getpass
from langchain_google_genai import ChatGoogleGenerativeAI #BaseChatModel
load_dotenv()  # Loads environment variables from .env file
# ignore all warnings in this notebook
import warnings
warnings.filterwarnings("ignore")

/workspaces/Langchain-Tutorial/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    os.environ["GOOGLE_API_KEY"] = google_api_key
    print("GOOGLE_API_KEY found in .env file.")
else:
    print("GOOGLE_API_KEY not found in .env file.")
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter GEMINI API key: ")

GOOGLE_API_KEY not found in .env file.


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

E0000 00:00:1761973080.998983   25230 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [6]:
from langchain.tools import tool

In [7]:
@tool
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.
    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"


In [8]:
print(search_database.name)

search_database


In [9]:
print(search_database.description)

Search the customer database for records matching the query.
    Args:
        query: Search terms to look for
        limit: Maximum number of results to return


In [10]:
@tool("web_search")
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.
    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"


In [11]:
search_database.name

'web_search'

In [12]:
@tool("web_search",description = "performs DB search using any type of query")
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.
    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"


In [13]:
search_database.description

'performs DB search using any type of query'

In [7]:
# pydantic schema
from pydantic import BaseModel, Field
from typing import Literal

class WeatherInput(BaseModel):
    """Input for weather queries."""
    location: str = Field(description="City name or coordinates")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit preference"
    )
    include_forecast: bool = Field(
        default=False,
        description="Include 5-day forecast"
    )

In [16]:
# json schema
weather_schema = {
    "type": "object",
    "properties": {
        "location": {"type": "string"},
        "units": {"type": "string"},
        "include_forecast": {"type": "boolean"}
    },
    "required": ["location", "units", "include_forecast"] }

In [44]:
weather_schema

{'type': 'object',
 'properties': {'location': {'type': 'string'},
  'units': {'type': 'string'},
  'include_forecast': {'type': 'boolean'}},
 'required': ['location', 'units', 'include_forecast']}

In [8]:
@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result


In [9]:
result = get_weather.invoke({"location": "Boston", "units": "fahrenheit", "include_forecast": True})
print(result)

Current weather in Boston: 72 degrees F
Next 5 days: Sunny


In [11]:
model_with_tools = model.bind_tools([get_weather])

In [28]:
response = model_with_tools.invoke("whats the weather in Boston in Celsius")

In [ ]:
print(response.content)
print("\nTool calls:")
for tool_call in response.tool_calls:
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")




Tool calls:
Tool: get_weather
Args: {'location': 'Boston', 'include_forecast': False, 'units': 'Celsius'}


In [30]:
# Step 1: Initial message
messages = [{"role": "user", "content": "What's the weather in Boston in Celsius?"}]
# Step 2: Model generates tool calls
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)
print("AI wants to call:", ai_msg.tool_calls)
# Step 3: Execute the tool
for tool_call in ai_msg.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)
    print(f"Tool returned: {tool_result}")
# Step 4: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(f"\nFinal answer: {final_response.content}")


AI wants to call: [{'name': 'get_weather', 'args': {'include_forecast': False, 'units': 'Celsius', 'location': 'Boston'}, 'id': 'ef4ac2f7-625f-44b9-9fde-e48f2eca1f9c', 'type': 'tool_call'}]
Tool returned: content='Current weather in Boston: 72 degrees C' name='get_weather' tool_call_id='ef4ac2f7-625f-44b9-9fde-e48f2eca1f9c'

Final answer: The weather in Boston is currently 72 degrees C.


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads environment variables from .env file

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
else:
    print("OPENAI_API_KEY not found in .env file.")
os.environ["SSL_NO_VERIFY"] = "1"
os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1"


In [18]:
from langchain.chat_models import init_chat_model
model = init_chat_model("openai:gpt-4.1-mini")
model_with_tools = model.bind_tools([get_weather])


In [ ]:
from langchain.chat_models import init_chat_model
model = init_chat_model("openai:gpt-4.1-mini")
tool = {"type": "web_search"}
model_with_tools = model.bind_tools([tool])
response = model_with_tools.invoke("What was a positive news story from today?")
response.content_blocks

In [36]:
# Force use of specific tool
model_forced = model.bind_tools([get_weather], tool_choice="get_weather")
response = model_forced.invoke("Tell me about Boston")
print(response.tool_calls)

[{'name': 'get_weather', 'args': {'location': 'Boston', 'units': 'metric', 'include_forecast': False}, 'id': 'call_MOH1YaURcQDRWTBMHo2J1SHx', 'type': 'tool_call'}]


In [ ]:
model_forced = model.bind_tools([get_weather,search_database], tool_choice="any")


In [13]:
response = model_with_tools.invoke(
    "What's the weather in Boston and Tokyo ?"
)
print(f"Number of tool calls: {len(response.tool_calls)}")

for tool_call in response.tool_calls:
    print(f"  - {tool_call['name']}: {tool_call['args']}")

results = []
for tool_call in response.tool_calls:
    result = get_weather.invoke(tool_call)
    results.append(result)
    print(f"Result: {result}")


Number of tool calls: 2
  - get_weather: {'location': 'Boston'}
  - get_weather: {'location': 'Tokyo'}
Result: content='Current weather in Boston: 22 degrees C' name='get_weather' tool_call_id='bb8b8fea-61b6-4ea4-92b5-1e4bbf5d0fb3'
Result: content='Current weather in Tokyo: 22 degrees C' name='get_weather' tool_call_id='487f5fdd-df37-49d8-a927-478b7455c775'


In [14]:
model_sequential = model.bind_tools([get_weather], parallel_tool_calls=False)

In [20]:
print("Streaming tool calls...")
for chunk in model_with_tools.stream("What's the weather in Boston and Tokyo?"):
    for tool_chunk in chunk.tool_call_chunks:
        if name := tool_chunk.get("name"):
            print(f"  Tool: {name}")
        if args := tool_chunk.get("args"):
            print(f"  Args chunk: {args}")


Streaming tool calls...
  Tool: get_weather
  Args chunk: {"lo
  Args chunk: catio
  Args chunk: n": "B
  Args chunk: osto
  Args chunk: n"}
  Tool: get_weather
  Args chunk: {"lo
  Args chunk: catio
  Args chunk: n": "T
  Args chunk: okyo
  Args chunk: "}


In [21]:
# Accumulate chunks into complete tool calls
gathered = None
for chunk in model_with_tools.stream("What's the weather in Boston?"):
    gathered = chunk if gathered is None else gathered + chunk
print("\nFinal accumulated tool calls:")
print(gathered.tool_calls)



Final accumulated tool calls:
[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_FDi5n4taJFYrkYWgPe7cQ9B4', 'type': 'tool_call'}]
